# Full Pipeline (on Tileset7) - Aug 2017
Created:  21 Aug 2018 <br>
Last update: 29 Aug 2018


### Goal: Combine the relevant steps from data import to unsupervised learning 

Many functions have gradually been developed in the prior notebooks (and added to 'imgutils'). In this notebook, the steps will be combined without all the intermediate analysis.


<hr>
## 1. Imports

In [ ]:
# this will remove warnings messages
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

# import
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score

import imgutils

In [ ]:
# Re-run this cell if you altered imgutils
import importlib
importlib.reload(imgutils)

<hr>
## 2. Data Definitions & Feature Specification

In [ ]:
# Data:
datafolder = '../data/Crystals_Apr_12/Tileset6_subset_1K'
n_tiles_x = 3  # mostly for visualization
n_tiles_y = 3


# Features to use:
#feature_funcs = [imgutils.img_mean, imgutils.img_std, imgutils.img_median, 
#                 imgutils.img_mode,
#                 imgutils.img_kurtosis, imgutils.img_skewness]
feature_funcs = [imgutils.img_std, imgutils.img_relstd, imgutils.img_mean, 
                 imgutils.img_skewness,  imgutils.img_kurtosis, imgutils.img_mode, imgutils.img_range]
feature_names = imgutils.stat_names(feature_funcs)

# Size of the grid, specified as number of slices per image in x and y direction:
default_grid_x = 4
default_grid_y = default_grid_x

<hr>
## 3. Import Data & Extract Features

In [ ]:
# image import:
print("Scanning for images in '{}'...".format(datafolder))
df_imgfiles = imgutils.scanimgdir(datafolder, '.tif')
imgfiles = list(df_imgfiles['filename'])
print("# of images: {} \n".format(len(imgfiles)))

# feature extraction:
print("Feature extraction...")
print("- Slicing up images in {} x {} patches. ".format(default_grid_y, default_grid_x))
print("- Extract statistics from each slice: {} ".format(', '.join(feature_names)))
print("...working...", end='\r')
df = imgutils.slicestats(imgfiles, default_grid_y, default_grid_x, feature_funcs)
print("# slices extracted: ", len(df))



<hr>
## 4. Machine Learning Pipeline

### Hyper parameters

In [ ]:
# data hyper-parameters
default_n_clusters = 3

# algorithm hyper-parameters:
kmeans_n_init = 10

In [ ]:
def run_ml_pipeline2(X, ml_name, ml_algorithm, standardize=True, use_pca=True, n_pca=None):
  
    # Setup algorithmic pipeline, including standardization
    pipeline = Pipeline([(ml_name, ml_algorithm)])
    
    # watch the order, pca should happen after scaling, but we insert at 0
    if (use_pca): 
        pipeline.steps.insert(0,('pca', PCA(n_components=n_pca)))
    if (standardize): 
        pipeline.steps.insert(0, ('scaling_{0}'.format(ml_name), StandardScaler()))
    
    # run the pipelines
    y = pipeline.fit_predict(X) # calls predict on last step to get the labels

    # report score:
    score = silhouette_score(X, y)
    
    return score, y

In [ ]:
def run_ml_pipelines2(df_data, feature_cols, n_clusters, standardize=True, use_pca=True, n_pca=None):
    global kmeans_n_init
    
    X = df_data.loc[:,feature_cols]
    
    # Setup ML clustering algorithms:    
    kmeans = KMeans(algorithm='auto', n_clusters=n_clusters, n_init=kmeans_n_init, init='k-means++')
    agglomerative =  AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='complete')  

    # run the pipelines
    print("Executing clustering pipelines...")
    score_kmeans, y_kmeans = run_ml_pipeline2(X, 'kmeans', kmeans, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    score_hier, y_hier = run_ml_pipeline2(X, 'hierarchical', agglomerative, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    print("Done\n")
    
    # collect data
    df_data['kmeans']=y_kmeans
    df_data['hierarchical']=y_hier

    # report results:
    print("\nClustering Scores:")
    print("K-means: ", score_kmeans)
    print("Hierarchical: ", score_hier)
       

In [ ]:
run_ml_pipelines2(df, feature_names, default_n_clusters, standardize=True, use_pca=True)

In [ ]:
df['dummy'] = 0
imgutils.show_large_heatmap(df, 'dummy', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(12,10))

In [ ]:
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(12,10))

In [ ]:
imgfile1 = imgfiles[0]
#df_heats1 = df[df['filename']==imgfile1]['kmeans']
#heats = np.reshape(df_heats1.values, (default_grid_y, default_grid_x))

subimgs, heats = imgutils.getimgslices_fromdf(df, imgfile1, 'kmeans')
heats = heats / np.max(heats)
imgutils.showheatmap(subimgs, heats, cmapname='Set1', heatdepend_opacity = False)

In [ ]:
n_patches = 40
df2 = imgutils.slicestats([imgfile1], n_patches, n_patches, feature_funcs)

In [ ]:
run_ml_pipelines2(df2, ['img_std', 'img_range'], 3, standardize=True, use_pca=True)

In [ ]:
imgutils.show_large_heatmap(df2, 'kmeans', [imgfile1], n_rows=1, n_cols=1, fig_size=(12,10))

In [ ]:
print(default_n_clusters)

### Why do these scores deviate from previous notebook?  Is there something going on with the sklearn pipeline or StandardScaler?


<hr>
## 5. Investigated scoring issue in other notebook

(see realxtals1-pipeline_scoring_issues)

Nothing was wrong with the pipeline or original step-by-step implementation. It was caused by different basis for the score calculation (pipeline impl. is using the original data while the step-by-step is using the normalized data for looking at the 'internal clustering score').

As such, turning normalization off (in the pipeline) gives higher scores. This does not mean the outcome is better, that needs visual inspection.

Adjusting the ml_pipeline to use silhouette scoring based on it's last transformation:
(later renamed the other ones to run_xxx2 to preserve them)

In [ ]:
from sklearn.metrics import calinski_harabaz_score

def run_ml_pipeline(X, ml_name, ml_algorithm, standardize=True, use_pca=True, n_pca=None):
  
    # Setup 'manual' pipeline (not using sklearn pipeline as intermediates are needed)
    feat_data = X
    if (standardize): 
        standardizer = StandardScaler()
        X_norm = standardizer.fit_transform(X)     
        feat_data = X_norm
    if (use_pca):  
        pca = PCA(n_components=n_pca)
        X_pca = pca.fit_transform(feat_data)
        feat_data = X_pca
    
    # run the pipelines
    y = ml_algorithm.fit_predict(feat_data) # calls predict oto get the labels

    # report score:
    #score = silhouette_score(feat_data, y)
    score = calinski_harabaz_score(feat_data,y)
    
    return score, y

In [ ]:
def run_ml_pipelines(df_data, feature_cols, n_clusters, standardize=True, use_pca=True, n_pca=None):
    global kmeans_n_init
    
    X = df_data.loc[:,feature_cols]
    
    # Setup ML clustering algorithms:    
    kmeans = KMeans(algorithm='auto', n_clusters=n_clusters, n_init=kmeans_n_init, init='k-means++')
    agglomerative =  AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='complete')  

    # run the pipelines
    print("Executing clustering pipelines...")
    score_kmeans, y_kmeans = run_ml_pipeline(X, 'kmeans', kmeans, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    score_hier, y_hier = run_ml_pipeline(X, 'hierarchical', agglomerative, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    print("Done\n")
    
    # collect data
    df_data['kmeans']=y_kmeans
    df_data['hierarchical']=y_hier

    # report results:
    print("\nClustering Scores:")
    print("K-means: ", score_kmeans)
    print("Hierarchical: ", score_hier)

In [ ]:
run_ml_pipelines(df, feature_names, default_n_clusters, standardize=True, use_pca=True)

More consistent with previous results and imo it is indeed better to assess the algorithm on how good it could cluster after all pre-processing

## 5. Visualize results

In [ ]:
run_ml_pipelines(df, feature_names, default_n_clusters, standardize=True, use_pca=True)
s = (8,6)
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)
imgutils.show_large_heatmap(df, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)

Run it again without PCA and/pr normalization compare results

In [ ]:
run_ml_pipelines(df, feature_names, default_n_clusters, standardize=True, use_pca=False)
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)
imgutils.show_large_heatmap(df, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)

In [ ]:
run_ml_pipelines(df, feature_names, default_n_clusters, standardize=False, use_pca=False)
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)
imgutils.show_large_heatmap(df, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)

### On this dataset, hierarchical works slightly better without normalization (!?!)

Hence, the method that runs two pipelines is less useful as they need different parameterization. So let's make two methods below that run the full pipeline including slicing

## 6. Combine import and pipeline:

In [ ]:
def import_data(imagefolder):
    df_imgfiles = imgutils.scanimgdir(imagefolder, '.tif')
    return list(df_imgfiles['filename'])  

def extract_features(imgfiles, feature_funcs, n_grid_rows, n_grid_cols):
    df = imgutils.slicestats(imgfiles, n_grid_rows, n_grid_cols, feature_funcs)
    feature_names = imgutils.stat_names(feature_funcs)
    return df, feature_names
    

In [ ]:
def run_kmeans_pipeline(df_data, feature_cols, n_clusters, standardize=True, use_pca=True, n_pca= None):
    global kmeans_n_init
   
    ml_name="kmeans"
    ml_algorithm = KMeans(algorithm='auto', n_clusters=n_clusters, n_init=kmeans_n_init, init='k-means++')

    X = df_data.loc[:,feature_cols]    
    score, y = run_ml_pipeline(X, ml_name, ml_algorithm, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    df_data[ml_name]= y

    return score

def run_hierarchical_pipeline(df_data, feature_cols, n_clusters, standardize=True, use_pca=True, n_pca=None):

    ml_name="hierarchical"
    ml_algorithm =  AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='complete')  

    X = df_data.loc[:,feature_cols]    
    score, y = run_ml_pipeline(X, ml_name, ml_algorithm, standardize = standardize, use_pca = use_pca, n_pca=n_pca)
    df_data[ml_name]= y

    return score

In [ ]:
def run_fullpipeline(imagefolder, n_image_rows, n_image_cols, 
                     n_grid_rows, n_grid_cols, feature_funcs, n_clusters, fig_size=(8,6), return_df = False):
    """
    Run the full pipeline from import to visualization.   
    """ 
    print("Working...\r")
    imgfiles = import_data(imagefolder)
    df, feature_names = extract_features(imgfiles, feature_funcs, n_grid_rows, n_grid_cols)
    print(feature_names)
    score_kmeans = run_kmeans_pipeline(df, feature_names, n_clusters, standardize=True, use_pca=True )
    score_hier = run_hierarchical_pipeline(df, feature_names, n_clusters, standardize=False, use_pca=False)

    print('Results:')
    print('Score k-means:', score_kmeans)
    print('Score hierarchical:', score_hier)
    
    print('Visualizing...')
    imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_image_rows, n_cols=n_image_cols, fig_size=fig_size)
    imgutils.show_large_heatmap(df, 'hierarchical', imgfiles, n_rows=n_image_rows, n_cols=n_image_cols, fig_size=fig_size)
    
    if return_df: return df
    

## 7. Try it out with different combinations of slices

### 4x4 - 3 clusters

In [ ]:

run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 4, 4, feature_funcs, 3)

### 6x6, 3 clusters

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 6, 6, feature_funcs, 3)

### 8x8, 3 clusters

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 8, 8, feature_funcs, 3)

<hr>

## 8. Try it out with different number of clusters

### 2 clusters (4x4 , 6x6, 8x8)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 4, 4, feature_funcs, 2)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 6, 6, feature_funcs, 2)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 8, 8, feature_funcs, 2)

Look again at hierarchical with scaling and pca on:

In [ ]:
imgfiles = import_data(datafolder)
df_temp, feature_names = extract_features(imgfiles, feature_funcs, 6, 6)
score = run_hierarchical_pipeline(df_temp, feature_names, 2, standardize=True, use_pca=True)
print("With scaling & pca:", score)
imgutils.show_large_heatmap(df_temp, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)    
score = run_hierarchical_pipeline(df_temp, feature_names, 2, standardize=False, use_pca=False)
print("Without scaling & pca:", score)
imgutils.show_large_heatmap(df_temp, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)   

In [ ]:
 imgfiles = import_data(datafolder)
df_temp, feature_names = extract_features(imgfiles, feature_funcs, 6, 6)
score = run_kmeans_pipeline(df_temp, feature_names, 2, standardize=True, use_pca=True)
print("With scaling & pca:", score)
imgutils.show_large_heatmap(df_temp, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)    
score = run_kmeans_pipeline(df_temp, feature_names, 2, standardize=False, use_pca=False)
print("Without scaling & pca:", score)
imgutils.show_large_heatmap(df_temp, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=s)   

Results with and without scaling / pca are pretty close actually.

### 4 clusters (4x4, 6x6, 8x8)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 4, 4, feature_funcs, 4)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 6, 6, feature_funcs, 4)

In [ ]:
run_fullpipeline(datafolder, n_tiles_y, n_tiles_x, 8, 8, feature_funcs, 4)

### Need 'grid search' versions to scan parameter space!!!

## 9. (Grid) Search for hyper-parameter optimizations

In [ ]:
imgfiles = import_data(datafolder)
df, feature_names = extract_features(imgfiles, feature_funcs, default_grid_y, default_grid_x)

### 9A. Number of clusters

In [ ]:
def optimize_kmeans_nclusters(df_imgstats, feature_names, n_clusters_start, n_clusters_end):
    """
    Run the full pipeline (except for visualization) for different number of clusters
    and show graph with score vs number of clusters.
    """   
    n_count = n_clusters_end + 1 - n_clusters_start                        
    result = np.empty(shape=(0,2), dtype=float)
                              
    for n in range(n_clusters_start, n_clusters_end+1):
        score = run_kmeans_pipeline(df, feature_names, n, standardize=True, use_pca=True )
        result = np.append(result, np.array([[n, score]]), axis=0)
    
    plt.plot(result[:,0], result[:,1])
    plt.title('K-means Clustering - Score vs. Number of Clusters ')
    plt.xlabel('# Clusters')
    plt.ylabel('Score')
    plt.show()
    


In [ ]:
def optimize_hierarchical_nclusters(df_imgstats, feature_names, n_clusters_start, n_clusters_end):
    """
    Run the full pipeline (except for visualization) for different number of clusters
    and show graph with score vs number of clusters.
    """  
    n_count = n_clusters_end + 1 - n_clusters_start                        
    result = np.empty(shape=(0,2), dtype=float)
                              
    for n in range(n_clusters_start, n_clusters_end+1):
        score = run_hierarchical_pipeline(df, feature_names, n, standardize=False, use_pca=False )
        result = np.append(result, np.array([[n, score]]), axis=0)
    
    plt.plot(result[:,0], result[:,1])
    plt.title('Hierarchical Clustering - Score vs. Number of Clusters ')
    plt.xlabel('# Clusters')
    plt.ylabel('Score')
    plt.show()


In [ ]:
optimize_kmeans_nclusters(df, feature_names, 2, 6)

In [ ]:
optimize_hierarchical_nclusters(df, feature_names, 2, 6)

### 9B. Number of features / PCA Components

In [ ]:
def optimize_kmeans_pcacomponents(df_imgstats, feature_names, n_clusters):
    """
    Run the full pipeline (except for visualization) for different number of PCA componenents 
    and show graph with the score vs number of components.
    """       
    n_count = len(feature_names) - 1            
    result = np.empty(shape=(0,2), dtype=float)                              
    
    for n in range(1, len(feature_names)):
        score = run_kmeans_pipeline(df, feature_names, n_clusters, standardize=True, use_pca=True, n_pca = n )
        result = np.append(result, np.array([[n, score]]), axis=0)                 
                   
    plt.plot(result[:,0], result[:,1])
    plt.title('K-means Clustering - Score vs. Number of PCA components ')
    plt.xlabel('N PCA Components')
    plt.ylabel('Score')
    plt.show()

In [ ]:
def optimize_hierarchical_pcacomponents(df_imgstats, feature_names, n_clusters):
    """
    Run the full pipeline (except for visualization) for different number of PCA componenents 
    and show graph with the score vs number of components.
    """             
    n_count = len(feature_names) - 1            
    result = np.empty(shape=(0,2), dtype=float)                              
    
    for n in range(1, len(feature_names)):
        score = run_hierarchical_pipeline(df, feature_names, n_clusters, standardize=True, use_pca=True, n_pca = n )
        result = np.append(result, np.array([[n, score]]), axis=0)                 
                   
    plt.plot(result[:,0], result[:,1])
    plt.title('Hierarchical Clustering - Score vs. Number of PCA components ')
    plt.xlabel('N PCA Components')
    plt.ylabel('Score')
    plt.show()

In [ ]:
optimize_kmeans_pcacomponents(df, feature_names, n_clusters=3)

Interesting. This graph says that a single component (after PCA transform) would already do a good job. Let's visualize

In [ ]:
score = run_kmeans_pipeline(df, feature_names, 3, standardize=True, use_pca=True, n_pca=1)
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(10,8))

Indeed not bad. Maybe not so surprise as in early EDA we saw that on this particular set the standard deviation is very informative (we used it for manual labelling)

A 'manual grid search' showed for 4 clusters the same

In [ ]:
optimize_kmeans_pcacomponents(df, feature_names, 4)
score = run_kmeans_pipeline(df, feature_names, 4, standardize=True, use_pca=True, n_pca=1)
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(10,8))

Actually this looks very good! It properly grouped 'fully occupied grid cells' from 'partially occupied'!

Let's also look at hierarchical clustering (although scores indicated better results without PCA)

In [ ]:
optimize_hierarchical_pcacomponents(df, feature_names, 2)
optimize_hierarchical_pcacomponents(df, feature_names, 3)
optimize_hierarchical_pcacomponents(df, feature_names, 4)

Don't know what these graphs really say; as the score is assessing cluster cohesion based on the last transformation in the pre-processing, it can be expected that less components are more separated. Maybe need to look at the 'elbows' here?

In [ ]:
# visualize for 4 clusters with 1 pca component:

In [ ]:
score = run_hierarchical_pipeline(df, feature_names, 4, standardize=True, use_pca=True, n_pca=1)
imgutils.show_large_heatmap(df, 'hierarchical', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(10,8))

Still looks like pretty good grouping into 4 clusters!

## 10. (Grid) Search for number of patches (expensive)

As the image slicing and feature extraction are the computationally expensive parts, both algorithms will be run in the search

### 10A. Optimize and plot graphs

In [ ]:
import sys

def optimize_slices(imagefolder, feature_funcs, n_clusters, list_n_slices):
    """
    Run the full pipeline (without visualization) for different number of image slices 
    and show graph of score vs number of slices.    
    Note that same number of slices is used in x and y direction.
    Returns a list of tuples of form (dataframe, n_slices, score_kmeans, score_hierarchical)  
    """ 
    imgfiles = import_data(imagefolder)
    results = []  

    print("Running slice optimization for {} clusters:".format(n_clusters))
    counter=0
    for n_slices in list_n_slices:
        progress = 100 * counter / len(list_n_slices)
        sys.stdout.write("{:.1f} %".format(progress) + "\r")
        
        df, feature_names = extract_features(imgfiles, feature_funcs, n_grid_rows=n_slices, n_grid_cols=n_slices)
        score_kmeans = run_kmeans_pipeline(df, feature_names, n_clusters, standardize=True, use_pca=True )
        score_hier = run_hierarchical_pipeline(df, feature_names, n_clusters, standardize=False, use_pca=False)
        results.append((df, n_slices, score_kmeans, score_hier))
        
        counter += 1
    
    sys.stdout.write("Done.")
    #sys.stdout.flush()
    
    plotdata = list(zip(*results))

    plt.plot(plotdata[1], plotdata[2], label="k-means (with PCA)")
    plt.plot(plotdata[1], plotdata[3], label="hierarchical")
    plt.title('Clustering - Score vs. Number of Image Slices')
    plt.xlabel('Number of slices')
    plt.ylabel('Score')
    plt.legend(loc='upper right')
    plt.show()
    
    return results



In [ ]:
slice_opt_results_3clusters = optimize_slices(datafolder, feature_funcs, n_clusters=3, list_n_slices=[2,4,8,10,12,16])

In [ ]:
slice_opt_results_4clusters = optimize_slices(datafolder, feature_funcs, n_clusters=4, list_n_slices=[2,4,8,10,12,16,20,32])

Interesting, optimum number of slices is around 10 for 3 clusters, and even higher number of patches for 4 clusters.

Also it is more efficient to do a ** grid search ** for clusters vs patches, as cluster evaluation is cheap while patches is expensive.

In [ ]:
import sys

def gridsearch_slices(imagefolder, feature_funcs, list_n_clusters, list_n_slices):
    """
    Run the full pipeline (without visualization) for different number of image slices 
    and show graph of score vs number of slices.    
    Note that same number of slices is used in x and y direction.
    Returns a list of tuples of form (dataframe, n_slices, n_clusters, score_kmeans, score_hierarchical)  
    """ 
    imgfiles = import_data(imagefolder)
    results = []  

    print("Running slice-#clusters grid-search...")
    counter=0
    for n_slices in list_n_slices:
        progress = 100 * counter / len(list_n_slices)
        sys.stdout.write("{:.1f} %".format(progress) + "\r")
        
        df, feature_names = extract_features(imgfiles, feature_funcs, n_grid_rows=n_slices, n_grid_cols=n_slices)
        
        for n_clust in list_n_clusters:
            df2 = df.copy()   # each result should have its own dataframe otherwise results are overwritten
            score_kmeans = run_kmeans_pipeline(df2, feature_names, n_clust, standardize=True, use_pca=True )
            score_hier = run_hierarchical_pipeline(df2, feature_names, n_clust, standardize=False, use_pca=False)
            results.append((df2, n_slices, n_clust, score_kmeans, score_hier))

        counter += 1
    
    sys.stdout.write("Done.")
    
    for n_clust in list_n_clusters:  
        results_for_cluster = [t for t in results if t[2]==n_clust]    
        plotdata = list(zip(*results_for_cluster))

        plt.plot(plotdata[1], plotdata[3], label="k-means (with PCA)")
        plt.plot(plotdata[1], plotdata[4], label="hierarchical")
        plt.title('{} Clusters - Score vs. Number of Image Slices'.format(n_clust))
        plt.xlabel('Number of slices')
        plt.ylabel('Score')
        plt.legend(loc='upper right')
        plt.show()
        
        
    return results

In [ ]:
slice_grid_search = gridsearch_slices(datafolder, feature_funcs, list_n_clusters=[2,3,4,5,6], list_n_slices=[2,4,6,8,10,12,14,16,20,32])

(I one time ran it up to 64 slices, but that take forever. k-means flattens and hierarchical goes drops drastically)

### 10B. Look at some heatmaps for 2 clusters

In [ ]:
def get_df_from_slicegridsearch(results, num_clust, num_slices):
    results_for_cluster = [t for t in results if t[2]==num_clust]    
    item_in_list = [t for t in results_for_cluster if t[1]==num_slices][0]
    return item_in_list[0] 

def show_from_slicegridsearch(results, num_clust, num_slices, n_img_rows, n_img_cols, heat_col='kmeans', fig_size=(12,10)):
    df_toshow = get_df_from_slicegridsearch(results, num_clust, num_slices)
    imgfiles = df_toshow['filename'].unique().tolist()
    extratitle = "{}x{} slices, {} clusters".format(num_slices,num_slices, num_clust)
    imgutils.show_large_heatmap(df_toshow, heat_col, imgfiles, n_rows=n_img_rows, n_cols=n_img_cols, fig_size=fig_size, subtitle=extratitle)

In [ ]:
def show_multiple_from_slicegridsearch(results, list_num_clust, list_num_slices, n_img_rows, n_img_cols, 
                                       heat_col='kmeans', fig_size=(12,10)):
    for n_slices in list_num_slices:
        for n_clust in list_num_clust:
            show_from_slicegridsearch(results, n_clust, n_slices, n_img_rows, n_img_cols, heat_col,  fig_size=fig_size)

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [2], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(10,8))

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [2], [2,4,8, 10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='hierarchical',  fig_size=(10,8))

### 10c. Look at some heatmaps for3 clusters

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [3], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(10,8))

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [3], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='hierarchical',  fig_size=(10,8))


### 10D. Look at some heatmaps for 4 clusters

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [4], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(10,8))

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [4], [10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(16,12))

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [4], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='hierarchical',  fig_size=(10,8))

### 10D. Look at some heatmaps for 5 & 6 clusters

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [5], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(10,8))

In [ ]:
show_multiple_from_slicegridsearch(slice_grid_search, [6], [2, 4, 8 ,10], n_img_rows=n_tiles_y, n_img_cols=n_tiles_x, heat_col='kmeans',  fig_size=(10,8))

## 11. Observations & Conclusions
* developed 'full pipeline' functionality
* looked at hyper-parameter optimization
* k-means works really well for 2 clusters
* for 4 clusters, hierarchical is better in sub-grouping the filled and partial filled

## 12.  Next steps:
* move some of the methods here into (new) module for re-use
* also add 'similarity learning' (see unsupervised2 notebook)
* apply the 'full pipeline' on the harder data set

Michael Janus, 29 August 2018 (1:50 am !)

# Try two-step pipeline
## step 1: filter out black tiles
## step 2: cluster remaining tiles

### Parametrize:

In [ ]:
n_clusters_step1 = 3
n_clusters_step2_kmeans = 3
n_clusters_step2_hierarchical = 3

n_patches_x = 16
n_patches_y = 16

### Feature extract

In [ ]:
# reset
df = df.drop(columns=['kmeans'])
df2 = None
df3 = None

In [ ]:
imgfiles = import_data(datafolder)
df, feature_names = extract_features(imgfiles, feature_funcs, n_patches_y, n_patches_x)

### Step 1: filter-out black tiles

In [ ]:
_ = run_kmeans_pipeline(df, feature_names, n_clusters_step1, standardize=True, use_pca=True )

In [ ]:
imgutils.show_large_heatmap(df, 'kmeans', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(10,8))

In [ ]:
df['kmeans'].value_counts()

In [ ]:
# cat_select = 1  
# we know for this it's the biggest set
i_max_count = df['kmeans'].value_counts()
cat_select = i_max_count.index[0]
print(cat_select)

In [ ]:
df2 = df[df['kmeans']==cat_select]
df2.head(3)

In [ ]:
score_kmeans = run_kmeans_pipeline(df2, feature_names, n_clusters_step2_kmeans, standardize=True, use_pca=True )
score_hierarch = run_hierarchical_pipeline(df2, feature_names, n_clusters_step2_hierarchical, standardize=False, use_pca=False )

In [ ]:
df2['kmeans'].value_counts()

In [ ]:
df2['hierarchical'].value_counts()

In [ ]:
df2=df2.rename(columns = {'kmeans':'kmeans2', 'hierarchical':'hierarchical2'})

In [ ]:
df2.head(3)

In [ ]:
df3 = df.merge(df2, 'left')

In [ ]:
df3.head(3)

In [ ]:
df3['kmeans2'].fillna(value=-1, inplace=True)

In [ ]:
df3['hierarchical2'].fillna(value=-1, inplace=True)

In [ ]:
df3.head(3)

In [ ]:
df3['heats']=df3['kmeans2']+1

In [ ]:
df3['heats'].value_counts()

In [ ]:
df3['heats2']=df3['hierarchical2']+1

In [ ]:
df3['heats2'].value_counts()

In [ ]:
imgutils.show_large_heatmap(df3, 'heats', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(16,12))

In [ ]:
imgutils.show_large_heatmap(df3, 'heats2', imgfiles, n_rows=n_tiles_y, n_cols=n_tiles_x, fig_size=(16,12))

# ANother subset

In [ ]:
datafolder2 = '../data/Crystals_Apr_12/Tileset6_subset2_1K'
n_tiles2_x = 2  # mostly for visualization
n_tiles2_y = 2

In [ ]:
df = run_fullpipeline(datafolder2, n_tiles2_y, n_tiles2_x, 10, 10, feature_funcs, 2, fig_size=(16,12), return_df=True)

In [ ]:
df['dummy'] = 0

In [ ]:
imgfiles = df['filename'].unique()
imgutils.show_large_heatmap(df, 'dummy', imgfiles, n_rows=n_tiles2_y, n_cols=n_tiles2_x, fig_size=(16,12))